In [1]:
!git clone https://github.com/dariush-bahrami/character-tokenizer.git

Cloning into 'character-tokenizer'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 16 (delta 3), reused 12 (delta 3), pack-reused 0
Unpacking objects: 100% (16/16), 5.11 KiB | 653.00 KiB/s, done.


In [2]:
 !pip install transformers

In [3]:
import string
import sys
sys.path.append("/kaggle/working/character-tokenizer")
from charactertokenizer import CharacterTokenizer

chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)

In [4]:
example = "Привет"
tokens = tokenizer(example)
print(tokens)

{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [5]:
tokenizer.convert_ids_to_tokens(tokens['input_ids'])

['[CLS]', 'П', 'р', 'и', 'в', 'е', 'т', '[SEP]']

Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

1. Напишите класс для Dataset/Dataloder и азбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)
2. Попробуйте несколько моделей: Bert, Albert, Deberta. (3 балла)
Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

ФИО: Мушенко Егор Сергеевич

In [6]:
!git clone https://github.com/Koziev/NLP_Datasets.git

Cloning into 'NLP_Datasets'...
remote: Enumerating objects: 595, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 595 (delta 73), reused 166 (delta 67), pack-reused 423
Receiving objects: 100% (595/595), 1.12 GiB | 19.02 MiB/s, done.
Resolving deltas: 100% (253/253), done.
Updating files: 100% (129/129), done.


In [7]:
!unzip NLP_Datasets/Stress/all_accents.zip

Archive:  NLP_Datasets/Stress/all_accents.zip
  inflating: all_accents.tsv         


In [8]:
import pandas as pd
import numpy as np

device = 'cuda'
df = pd.read_csv('all_accents.tsv', delimiter='\t')
df.shape[0]

1680534

In [9]:
df['target_idx'] = [word.find('^')for word in df['-д^е']]
df.sample(10)

,-де,-д^е,target_idx
1040735,подзанялись,подзанял^ись,8
12959,адвокатским,адвок^атским,5
354236,дренаж,дрен^аж,4
1628003,читалками,чит^алками,3
900768,откатившею,откат^ившею,5
1328016,рокируйте,рокир^уйте,5
1400038,слякотный,сл^якотный,2
1490237,типологии,типол^огии,5
1274344,разочаровывающему,разочар^овывающему,7
942689,паломнической,пал^омнической,3


In [10]:
df.drop(index=np.where(df.target_idx == -1)[0], inplace=True)
df.shape[0]

1680027

In [11]:
train_words = df['-де']
target = df.target_idx

In [12]:
max_len = 0
for word in train_words:
    input_ids = tokenizer(word)['input_ids']
    max_len = max(max_len, len(input_ids))
print('Max word length: ', max_len)

Max word length:  58


In [13]:
import torch
from tqdm.notebook import tqdm

input_ids = []
attention_masks = []
MAX_LENGTH = max_len
# For every sentence...
for word in tqdm(train_words):
    encoded_dict = tokenizer(
                        word,
                        add_special_tokens = True,
                        max_length = MAX_LENGTH,   
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


  0%|          | 0/1680027 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [14]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(target.values)


In [15]:
i = 10000
print('Original: ', train_words[i])
print('Token IDs:', input_ids[i])

Original:  агарак
Token IDs: tensor([ 0,  8, 14,  8, 42,  8, 30,  8, 12,  8, 36,  1,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4])


In [16]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)
max_len = 58

In [17]:
train_size = int(0.5 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

840,013 training samples
840,014 validation samples


In [18]:
torch.cuda.empty_cache()

In [19]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 64

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size 
        )

In [20]:
import time
import datetime
import numpy as np

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [21]:
import random

def train_model(model, optimizer, scheduler, epochs, val_step=True):
    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    training_stats = []

    total_t0 = time.time()

    for epoch_i in range(0, epochs):
        t0 = time.time()
        total_train_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):
            if step % 100 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}'.format(step, len(train_dataloader)))

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            model.zero_grad()        

            output = model(b_input_ids, token_type_ids=None,
                          attention_mask=b_input_mask, labels=b_labels)

            total_train_loss += output.loss.item()
            output.loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)            
        training_time = format_time(time.time() - t0)
        print("\n  Avg loss: {0:.2f}".format(avg_train_loss))
            
        # val
        if not val_step:
            continue

        t0 = time.time()
        model.eval()

        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            with torch.no_grad():        
                output = model(b_input_ids, token_type_ids=None, 
                              attention_mask=b_input_mask, labels=b_labels)
                
            total_eval_loss += output.loss.item()

            logits = output.logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            total_eval_accuracy += flat_accuracy(logits, label_ids)
            

        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        avg_val_loss = total_eval_loss / len(validation_dataloader)
        validation_time = format_time(time.time() - t0)       
        print("  Avg validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Total validation time: {:}\n".format(validation_time))

In [22]:
def test_model(model):
    model.eval()
    samples = df.sample(10)
    pred = []
    with torch.no_grad():
        for i in range(10):
            word = samples.iloc[i]['-де']
            input = torch.tensor(tokenizer(word)['input_ids']).to(device)
            output = model(input.unsqueeze(0))
            preds = output.logits.detach().cpu().numpy()
            ans = np.argmax(preds, axis=1)[0]
            pred.append(word[:ans] + '^' + word[ans:])
    samples['prediction'] = pred
    samples['is_right'] = [int(inp == out) for inp, out in zip(samples['-д^е'], pred)]
    return samples

### BERT

In [23]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup


model = BertForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny", 
    num_labels = max_len - 1,   
    output_attentions = False, 
    output_hidden_states = False, 
)

model.to(device)
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [24]:
train_model(model, optimizer, scheduler, epochs)

  Batch   100  of  13,126
  Batch   200  of  13,126
  Batch   300  of  13,126
  Batch   400  of  13,126
  Batch   500  of  13,126
  Batch   600  of  13,126
  Batch   700  of  13,126
  Batch   800  of  13,126
  Batch   900  of  13,126
  Batch 1,000  of  13,126
  Batch 1,100  of  13,126
  Batch 1,200  of  13,126
  Batch 1,300  of  13,126
  Batch 1,400  of  13,126
  Batch 1,500  of  13,126
  Batch 1,600  of  13,126
  Batch 1,700  of  13,126
  Batch 1,800  of  13,126
  Batch 1,900  of  13,126
  Batch 2,000  of  13,126
  Batch 2,100  of  13,126
  Batch 2,200  of  13,126
  Batch 2,300  of  13,126
  Batch 2,400  of  13,126
  Batch 2,500  of  13,126
  Batch 2,600  of  13,126
  Batch 2,700  of  13,126
  Batch 2,800  of  13,126
  Batch 2,900  of  13,126
  Batch 3,000  of  13,126
  Batch 3,100  of  13,126
  Batch 3,200  of  13,126
  Batch 3,300  of  13,126
  Batch 3,400  of  13,126
  Batch 3,500  of  13,126
  Batch 3,600  of  13,126
  Batch 3,700  of  13,126
  Batch 3,800  of  13,126
  Batch 3,90

In [25]:
test_model(model)

,-де,-д^е,target_idx,prediction,is_right
331232,додумывавшими,дод^умывавшими,3,дод^умывавшими,1
647800,маринова,мар^инова,3,мар^инова,1
431832,зарядившимся,заряд^ившимся,5,заряд^ившимся,1
424960,заполучаем,заполуч^аем,7,заполуч^аем,1
1643854,шлихи,шлих^и,4,шл^ихи,0
442768,затечешь,затеч^ешь,5,затеч^ешь,1
563471,комплементе,комплем^енте,7,комплем^енте,1
819140,обмельчавшей,обмельч^авшей,7,обмельч^авшей,1
1653022,щурившим,щ^урившим,1,щ^урившим,1
1363772,сдружаетесь,сдруж^аетесь,5,сдруж^аетесь,1


### Albert

In [26]:
from transformers import AutoModelForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup


model = AutoModelForSequenceClassification.from_pretrained(
    "albert-base-v2", 
    num_labels = max_len - 1,   
    output_attentions = False,
    output_hidden_states = False, 
)

model.to(device)
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

epochs = 1
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [27]:
torch.cuda.empty_cache()

In [28]:
train_model(model, optimizer, scheduler, epochs, val_step=False)

  Batch   100  of  13,126
  Batch   200  of  13,126
  Batch   300  of  13,126
  Batch   400  of  13,126
  Batch   500  of  13,126
  Batch   600  of  13,126
  Batch   700  of  13,126
  Batch   800  of  13,126
  Batch   900  of  13,126
  Batch 1,000  of  13,126
  Batch 1,100  of  13,126
  Batch 1,200  of  13,126
  Batch 1,300  of  13,126
  Batch 1,400  of  13,126
  Batch 1,500  of  13,126
  Batch 1,600  of  13,126
  Batch 1,700  of  13,126
  Batch 1,800  of  13,126
  Batch 1,900  of  13,126
  Batch 2,000  of  13,126
  Batch 2,100  of  13,126
  Batch 2,200  of  13,126
  Batch 2,300  of  13,126
  Batch 2,400  of  13,126
  Batch 2,500  of  13,126
  Batch 2,600  of  13,126
  Batch 2,700  of  13,126
  Batch 2,800  of  13,126
  Batch 2,900  of  13,126
  Batch 3,000  of  13,126
  Batch 3,100  of  13,126
  Batch 3,200  of  13,126
  Batch 3,300  of  13,126
  Batch 3,400  of  13,126
  Batch 3,500  of  13,126
  Batch 3,600  of  13,126
  Batch 3,700  of  13,126
  Batch 3,800  of  13,126
  Batch 3,90

In [29]:
test_model(model)

,-де,-д^е,target_idx,prediction,is_right
331232,додумывавшими,дод^умывавшими,3,дод^умывавшими,1
647800,маринова,мар^инова,3,мар^инова,1
431832,зарядившимся,заряд^ившимся,5,заряд^ившимся,1
424960,заполучаем,заполуч^аем,7,заполуч^аем,1
1643854,шлихи,шлих^и,4,шл^ихи,0
442768,затечешь,затеч^ешь,5,затеч^ешь,1
563471,комплементе,комплем^енте,7,комплем^енте,1
819140,обмельчавшей,обмельч^авшей,7,обмельч^авшей,1
1653022,щурившим,щ^урившим,1,щ^урившим,1
1363772,сдружаетесь,сдруж^аетесь,5,сдруж^аетесь,1


### Deberta

In [30]:
from transformers import AutoModelForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup


model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-small", 
    num_labels = max_len - 1,   
    output_attentions = False, 
    output_hidden_states = False, 
)

model.to(device)
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

epochs = 1
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)


Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [31]:
train_model(model, optimizer, scheduler, epochs, val_step=False)

  Batch   100  of  13,126
  Batch   200  of  13,126
  Batch   300  of  13,126
  Batch   400  of  13,126
  Batch   500  of  13,126
  Batch   600  of  13,126
  Batch   700  of  13,126
  Batch   800  of  13,126
  Batch   900  of  13,126
  Batch 1,000  of  13,126
  Batch 1,100  of  13,126
  Batch 1,200  of  13,126
  Batch 1,300  of  13,126
  Batch 1,400  of  13,126
  Batch 1,500  of  13,126
  Batch 1,600  of  13,126
  Batch 1,700  of  13,126
  Batch 1,800  of  13,126
  Batch 1,900  of  13,126
  Batch 2,000  of  13,126
  Batch 2,100  of  13,126
  Batch 2,200  of  13,126
  Batch 2,300  of  13,126
  Batch 2,400  of  13,126
  Batch 2,500  of  13,126
  Batch 2,600  of  13,126
  Batch 2,700  of  13,126
  Batch 2,800  of  13,126
  Batch 2,900  of  13,126
  Batch 3,000  of  13,126
  Batch 3,100  of  13,126
  Batch 3,200  of  13,126
  Batch 3,300  of  13,126
  Batch 3,400  of  13,126
  Batch 3,500  of  13,126
  Batch 3,600  of  13,126
  Batch 3,700  of  13,126
  Batch 3,800  of  13,126
  Batch 3,90

In [32]:
test_model(model)

,-де,-д^е,target_idx,prediction,is_right
331232,додумывавшими,дод^умывавшими,3,дод^умывавшими,1
647800,маринова,мар^инова,3,мар^инова,1
431832,зарядившимся,заряд^ившимся,5,заряд^ившимся,1
424960,заполучаем,заполуч^аем,7,заполуч^аем,1
1643854,шлихи,шлих^и,4,шл^ихи,0
442768,затечешь,затеч^ешь,5,зат^ечешь,0
563471,комплементе,комплем^енте,7,комплем^енте,1
819140,обмельчавшей,обмельч^авшей,7,обмельч^авшей,1
1653022,щурившим,щ^урившим,1,щур^ившим,0
1363772,сдружаетесь,сдруж^аетесь,5,сдруж^аетесь,1
